In [0]:
%run "./Configuration"

In [0]:
# Define the metadata widget:
dbutils.widgets.text("entityName", "")
dbutils.widgets.text("fileLocation", "")
dbutils.widgets.text("entityColumns", "")

# Save the parameter to a variable:
entityName =    dbutils.widgets.get('entityName')
file_location = dbutils.widgets.get('fileLocation')[66:]
columns =       dbutils.widgets.get('entityColumns')

# Replace uppercase False/True with lowercase:
columns = columns.replace('False', 'false').replace('True', 'true')

# Define location of the raw data:
parquet_path = RAW + f"{file_location}"

In [0]:
if entityName in ['customers', 'products']:
    # Load the column data as a JSON:
    json_cols = json.loads(columns)

    # Dictionary to map column data types to spark data types:
    data_type = {'int': 'integer', 'varchar': 'string', 'double': 'double', 'date': 'date'}

    # Read RAW data with schema:
    df = spark.read.parquet(parquet_path)

    # Iterate through columns and cast correct data type:
    for col in json_cols:
        df = df.withColumn(col['columnName'], df[col['columnName']].cast(data_type[col['dataType']['name']]))

In [0]:
# Check there is data in the dataframe
if df.count() > 0:
    # Define path to write dataframe to:
    transform_path = TRANSFORM +  f"{file_location}"
    
    # Write dataframe:
    df.write.mode('overwrite').parquet(transform_path)

    # Output file_path:
    dbutils.notebook.exit(transform_path)

